In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Activation, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow.keras.backend as k
import cv2

Using TensorFlow backend.


In [2]:
num_classes = 5 #5 celebrities
batch_size = 128 # images in one go

datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)
traindata = datagen.flow_from_directory(directory="celeb",
                                        subset='training',
                                        target_size=(128, 128),
                                        batch_size=batch_size,
                                        class_mode='categorical')
testdata = datagen.flow_from_directory(directory="celeb",
                                       subset='validation',
                                       target_size=(128, 128),
                                       batch_size=batch_size,
                                       class_mode='categorical')

Found 32807 images belonging to 5 classes.
Found 8199 images belonging to 5 classes.


In [3]:
#Clear session to cleanup older models
k.clear_session()

#create model
model = Sequential()

# First CONV-ReLU Layer
model.add(Conv2D(input_shape=(128,128,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(BatchNormalization())

# Second CONV-ReLU Layer
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(BatchNormalization())

# Max Pooling with Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))
          
# 3rd set of CONV-ReLU Layers         
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())

# 4th Set of CONV-ReLU Layers
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())

# Max Pooling with Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# 5th Set of CONV-ReLU Layers
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())

# 6th Set of CONV-ReLU Layers
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())

# Max Pooling with Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# 7th Set of CONV-ReLU Layers
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())

# 8th Set of CONV-ReLU Layers
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(BatchNormalization())

# Max Pooling with Dropout
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# First set of FC or Dense Layers
model.add(Flatten())
model.add(Dense(units=512,activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Second set of FC or Dense Layers
model.add(Dense(units=512,activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Final Dense Layer
model.add(Dense(units=num_classes, activation="softmax"))

#Compile the model
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

#Print model Summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)      

In [4]:
#Define Checkpoints and early stopping
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("m_cnn_1.h5", 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             period=1)

early = EarlyStopping(monitor='val_accuracy', 
                      min_delta=0, 
                      patience=3, 
                      verbose=1, 
                      mode='auto')

#Fit the model
hist = model.fit_generator(steps_per_epoch=int(32807//batch_size), #Number of training images//batch size
                           generator=traindata, 
                           validation_data= testdata, 
                           validation_steps=int(8199//batch_size), #Number of test images//batch size,
                           epochs=10,
                           callbacks=[checkpoint,early])

Epoch 1/10
  5/256 [..............................] - ETA: 6:09:11 - loss: 2.2738 - accuracy: 0.3187

KeyboardInterrupt: 